In [1]:
## this script will be used to calculate k-fold to determine how many trials we need

####### to run this you will want to:
# 1. update the file to be your data file
# 2. update the trial increments
# 3. update the location of the file you are writing to

# imports and reading csv
import pandas as pd
import numpy as np
k_data = pd.read_csv(r"C:\Users\allym\Documents\MQP\all_data\data_ally_50000_54999.csv")


In [2]:
# initializations (can change some of these if you want)
k = 5
sets_of_trials = [[50000,54999]]


In [3]:
# this will make it automatically loop through the sets of trials
index = k_data.index
for [m,n] in sets_of_trials:
    x = m
    seed = k_data['seed']
    conditionm = seed == m
    seedm_index = index[conditionm]
    conditionn = seed == n
    seedn_index = index[conditionn]
    current_seed_index = seedm_index
    while ((current_seed_index<=seedn_index) and (current_seed_index>=seedm_index)):

        # shuffle and split the data into k random groups
        # we do this 100 times to get 100 trials (possible that some of them will be the same)
        # can change the number of trials
        i = 0
        total_good_trials = 0
        total_bad_trials = 0

        while (i < 100):
            good_enough_trials = 0
            bad_more_trials = 0
            shuffled = k_data.sample(frac=1)
            result = np.array_split(shuffled,k)

            # for each group, calculate the average of the evaluation metrics
            for part in result:
                other_eval_sum = 0
                eval_sum = sum(part['evaluation_metric'])
                group_avg = eval_sum/len(part['evaluation_metric'])
        #         print('group average: ',group_avg)

            # calculate the average of the remaining parts
                for other_part in result:
                    if (other_part is not part):
                        other_eval_sum = other_eval_sum + sum(other_part['evaluation_metric'])
                other_avg = other_eval_sum/((k-1)*len(part['evaluation_metric']))
        #         print('other average', other_avg)

            # calculate the percent difference between these values for each
            # if the percent difference is less than 5%, it's good, otherwise, it's bad
            # so add it to the appropriate counter
                percent_difference = abs((group_avg - other_avg)/other_avg)
        #         print('percent difference',percent_difference)
                if (percent_difference <= 0.05):
                    good_enough_trials += 1
                    total_good_trials += 1
                elif (percent_difference > 0.05):
                    bad_more_trials += 1
                    total_bad_trials += 1

        #     print('i',i,'good: ' ,good_enough_trials,'bad: ', bad_more_trials)

            i = i + 1
        x += 1
        current_seed_index += 1

        # total number of trials that are good and bad
        # calculates the percentage of good trials
        # when the percentage of good trials is 95% or greater, we are happy
    print([m,n],'total good trials: ',total_good_trials, 'total bad trials', total_bad_trials)
    percent_good_trials = (total_good_trials/(total_good_trials+total_bad_trials))*100
    print([m,n], percent_good_trials)

[50000, 54999] total good trials:  454 total bad trials 46
[50000, 54999] 90.8
